In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, when
import unicodedata
import os


In [9]:
spark = SparkSession.builder \
    .appName("LandingToBronze") \
    .getOrCreate()

In [10]:
landing_path = "../../datalake/landing"
bronze_path = "../../datalake/bronze"

In [11]:
colunas_historico = [
    "ano", "mes", "agente_regulado", "codigo_produto", "nome_produto",
    "regiao_origem", "uf_origem", "regiao_destino", "uf_destino",
    "destino_consumo", "volume_vendido"
]

In [12]:
class datanormalizer:
    @staticmethod
    def padronizar_nomes(df):
        coluna_limpa = [
            unicodedata.normalize('NFKD', c.lower().strip())
            .encode('ascii', 'ignore')
            .decode('utf-8')
            .replace(' ', '_')
            for c in df.columns
        ]
        for old, new in zip(df.columns, coluna_limpa):
            df = df.withColumnRenamed(old, new)
        return df


In [13]:
class data_cleaner:
    @staticmethod
    def limpeza_basica(df):
        df = df.dropDuplicates()
        df = df.na.drop(how='all')
        for col_name in df.columns:
            if "data" in col_name or "mes" in col_name:
                df = df.withColumn(col_name,
                    when(col(col_name).rlike(r"^\d{4}-\d{2}-\d{2}$"), to_date(col(col_name), "yyyy-MM-dd"))
                    .otherwise(None))
        return df

In [14]:
for arquivo in os.listdir(landing_path):
    if arquivo.endswith(".csv"):
        caminho_arquivo = os.path.join(landing_path, arquivo)
        nome_base = os.path.splitext(arquivo)[0]
        destino = os.path.join(bronze_path, nome_base)

        if nome_base == "Liquidos_Vendas_Historico_2007_a_2017":
            df = spark.read.option("header", False)\
                           .option("encoding", "iso-8859-1")\
                           .option("delimiter", ";")\
                           .csv(caminho_arquivo)
            df = df.toDF(*colunas_historico)
        else:
            df = spark.read.option("header", True)\
                           .option("encoding", "iso-8859-1")\
                           .option("delimiter", ";")\
                           .csv(caminho_arquivo)
        
        df = datanormalizer.padronizar_nomes(df)
        df = data_cleaner.limpeza_basica(df)

        df.write.mode("overwrite").parquet(destino)

25/06/17 17:56:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/06/17 17:56:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/06/17 17:56:36 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
